# Reinforcement-Learning Reference
Reference Implementation for Reinforcement Assignment**

### Source:
These are the files required to build your reinforcement learning algorithm. 

- [common.py](common.py) with constants
- [util.py](util.py) with util functions
- [game.py](game.py) with drawing calls
- [environment.py](environment.py) contains the scenario behavior
- [agent.py](agent.py) contains training components, such as environment interaction and previous state

### Assignment
The goal of this assignment is to implement the core of the Q-Learning algorithm. You will be responsible for implementing three distinct methods:
- The exploration function(**f()** method)
- The Q-Learning update method (**get_action()** method)
- Implemnet a decreasing function for the learning rate (**alpha()** method)

In this scenario we help the agent, via reinforcement learning, to navigate and maximise rewards within a map, aiming to reach the move between an initial state and the goal state, represented by a treasure chest. In some scenarios there will be a rupee that the agent can gather. The rewards are +50 for reaching the chest, +40 for getting the rupee and -1 for any other tile.

This assignment is not graded. Thus no tests are provided.

### Execution
The execution of this assignment can be done entirely in this Jupyter Notebook, or in two distinct python files. If you want to program/test outside of Jupyter just follow these instructions. Please note that currently there is a problem with pygame (a package responsible for displaying the agent moving in the environment) and Jupyter, which causes the jupyter kernel to crash after closing the pygame window. Instead of closing the window, press **space bar** to close the pygame window. To install pygame, run:

```
pip install pygame --user
```

In order to test your code and get the convergence episode, you can use the environment.py file:
```
python environment.py [Map]
```

To check the converged solution of your algorithm, you can run the GUI to see the agent executing the learned policy in each map.
```
python game.py [Map]
```

### Implementation

In this file, we provide the basic architecture to build your Q-Learning algorithm. Additionally, you can implement your algorithm in the [link_ref.py](link_ref.py) file, if you wish to work outside of Jupyter Notebook. 



## Exploration function
Implement the optimistic estimate function described by the following equation.
$$
f(u,n) = \begin{cases}
					R^{+} & \mathit{if} \text{ }n < N_{e} \\
					u & \mathit{otherwise}
				   \end{cases}
$$
Consider that instead of just using the utility *u* in the python method as parameter, we are using **qv**, which is a Q-Value (state-action pair). The parameter **self** here is a instance of the Link class. You can check the frequency of a Q-Value using:
```
self.frequency[qv]
```
**Note:** If the Q-Value *qv* was never visited, it will not be inside the frequency table.


In [1]:
#Receives a q-value and returns a utility
def f(self, qv):
    from random import randint
    return randint(0,50)
   

## Q-Learning
Implement the Q-Learning update equation. Consider the learning rate as a fixed value for now. Again, **self** refers to an instance of the Link class. 
You can check the reward of a state with:
```
reward_prime = self.env.state_reward((state.x, state.y))
```
To check if a state is terminal, we use this:
```
if self.env.terminal((state.x, state.y)):
```
Finally, when creating a entry on the **q-table** with the **None** action we use:
```
 self.q_values[qvalue.QValue(state, NO_OP)] = reward_prime
```
<img src="https://user-images.githubusercontent.com/4201145/45648565-4d0c0580-ba9f-11e8-82fd-1a4f127c1959.png" width="70%" height="70%"/>

The ***argmax*** and ***max*** functions are already implemented as ***argmax_a(s)*** and ***max_a(s)***.

In [6]:
def get_action(self, state):
    """This function corresponds to the q-learning-agent(percept) 
    function in Russel and Norvig's book
    """
    if self.env.terminal((state.x, state.y)):
        if QValue(state, 'NO_OP') not in self.frequency:
            self.frequency[QValue(state, 'NO_OP')] = 0
        self.frequency[QValue(state, 'NO_OP')] += 1
        self.q_values[QValue(state, 'NO_OP')] = self.env.state_reward((state.x, state.y))
    if self.p_state is not None:
        qv = QValue(self.p_state,self.p_action)
        if qv not in self.frequency:
            self.frequency[qv] = 0
        self.frequency[qv] += 1
        q_sa = self.q_value_check(qv)
        #Implement Q-Learning update here
        #self.q_values[qv] = update rule here
    self.p_state = state
    self.p_action = self.argmax_a(state)
    self.p_reward = self.env.state_reward((state.x, state.y))
    return self.p_action

# Learning rate
Choose a decreasing function based on how many times a state has been visited and implement it here. Again, **self** refers to an instance of the Link class.

In [3]:
def alpha(self,qv):
    # Implement here a more sophisticated learning rate
    return 0.9

# Base code
Base code of the Link class. You don't need to implement anything here. The attributes of this class are:
- self.q_values -> A dictionary that maps a Q-Value to a utility.
- self.frequency -> A dictionary that maps a Q-Value to the number of times it has been visited.
- self.state -> The actual state (used for internal methods, don't need to use this on the Q-Learning implementation)
- self.reward -> The actual reward (used for internal methods, don't need to use this on the Q-Learning implementation)
- self.action -> The actual action (used for internal methods, don't need to use this on the Q-Learning implementation)
- self.p_state -> The previous state.
- self.p_reward -> The previous reward.
- self.p_action -> The previous action.
- self.gamma -> Discount factor.
- self.r_plus -> The highest reward in the environment.
- self.exploration -> The exploration threshold.
- self.env -> A instance of the environment (internal use).
- self.prev_qtable -> Previous Q-Table, used for checking the convergence (internal use).

In [4]:
#!/usr/bin/env python
# Four spaces as indentation [no tabs]
# Standard Q-Learning implementation.
import math, copy, random, logging
import numpy as np
from qvalue import *
from common import *
from util import *
from agent import *

class Link(Agent):

    def __init__(self):

        Agent.__init__(self)
        self.q_values = dict()
        self.frequency = dict()
        self.state = None
        self.reward = None
        self.action = NO_OP
        self.p_state = None 
        self.p_reward = None
        self.p_action = None
        self.gamma = 0.9
        self.r_plus = 50
        self.exploration = 1
        self.env = None
        self.prev_qtable = dict()

    def reset(self, env):
        """
        Reset the state to the initial environment state
        """
        self.state = env.init

    def train(self, env):
        """
        Execute MAX_TRAINING_EPISODES rounds or until converge.
        """
        print('It will converge at', CONVERGENCE_THRESHOLD)

        self.reset(env)
        self.env = env

        executions = 0
        last_plan = []
        while executions < MAX_TRAINING_EPISODES:
            self.state = self.make_state(env)
            action = self.get_action(self.state)
            last_plan.append(action)
            self.env.execute(action)
            if env.terminal((self.state.x, self.state.y)):
                executions += 1
                
                self.p_state = self.p_action = self.p_reward = self.state = self.action = self.reward = None
                self.reset(env)

                if self.converged():
                    break
                else:
                    last_plan = []
                    self.prev_qtable = copy.deepcopy(self.q_values)

                #print('Episode', executions, ': convergence %', self.convergence)

        print('Episode' , executions, ' : converged at', self.convergence)
        print('Last plan executed: ', [ACTIONS_NAMES[x] for x in last_plan])

    def alpha(self, qv):
        """
        Alpha value, currently returning 0.9 because it converges pretty fast. 
        """
        return alpha(self,qv)

    def f(self, qv):
        """
        Exploration function. Use maxreward if the q_value was not explored.
        """
        return f(self,qv)

    def get_action(self, state):
        return get_action(self,state)   

    def q_value_check(self, qv):
        if qv in self.q_values:
            return self.q_values[qv]
        return 0.0

    def max_a(self, state):
        return np.max([self.q_value_check((QValue (state,action))) for action in self.env.available_actions((state.x, state.y))])
    
    def argmax_a(self, state):
        action_index = np.argmax([self.f(QValue (state,action)) for action in self.env.available_actions((state.x, state.y))])
        action = self.env.available_actions((state.x, state.y))[action_index]
        return action

    def make_state(self, env):
        """
        Build state using position and rupees.
        """
        return State(env.state[0], env.state[1], env.rupees)

    def return_qvalue(self, qvalue):
        if qvalue in self.q_values:
            return self.q_values[qvalue]
        return 0

    def converged(self):
        """
        Return True if the change between previous util table and current util table
        are smaller than the convergence_threshold.
        """
        self.convergence = self.convergence_metric()
        return self.convergence < CONVERGENCE_THRESHOLD

    def run(self, env):
        """
        Execute actions.
        """
        self.action = self.argmax_a(self.make_state(env))
        #print "Running action: ", ACTIONS_NAMES[self.action]
        self.state, self.reward = env.execute(self.action)
        return self.action, self.state


    def convergence_metric(self):
        """
        Return the convergence metric.
        """
        prev = sum(self.prev_qtable.values())
        curr = sum(self.q_values.values())
        return math.sqrt(abs(curr - prev))




# Train Agent
In this cell, we train the agent. You can change the map and add any code you like here.

In [5]:
try:
    from game import *
    pg = True
except ImportError:
    pg = False
    
from environment import *

logger = logging.getLogger()
sx, sy, map_data, map_width, map_height = read_map("maps/medium.txt")

agt = Link()

env = Environment(sx, sy, map_data, map_width, map_height)

start_time = time.time()
agt.train(env)
elapsed_time = time.time() - start_time
print('It took', elapsed_time,'seconds to train.' )
if pg:
    #Comment this line if you do not want to use the UI
    Game(env, agt)
    pass

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html
('It will converge at', 0.05)
('Episode', 2, ' : converged at', 0.0)
('Last plan executed: ', ['DOWN', 'UP', 'DOWN', 'UP', 'DOWN', 'DOWN', 'DOWN', 'UP', 'DOWN', 'UP', 'UP', 'UP', 'DOWN', 'UP', 'DOWN', 'DOWN', 'RIGHT', 'LEFT', 'UP', 'DOWN', 'RIGHT', 'LEFT', 'DOWN', 'UP', 'RIGHT', 'LEFT', 'RIGHT', 'LEFT', 'RIGHT', 'RIGHT', 'DOWN', 'UP', 'RIGHT', 'LEFT', 'LEFT', 'RIGHT', 'DOWN', 'UP', 'RIGHT', 'LEFT', 'RIGHT', 'LEFT', 'DOWN', 'DOWN', 'UP', 'DOWN', 'UP', 'DOWN', 'DOWN', 'UP', 'DOWN', 'UP', 'DOWN', 'UP', 'UP', 'UP', 'DOWN', 'DOWN', 'DOWN', 'UP', 'UP', 'UP', 'LEFT', 'LEFT', 'UP', 'DOWN', 'UP', 'UP', 'DOWN', 'DOWN', 'DOWN', 'UP', 'RIGHT', 'RIGHT', 'LEFT', 'LEFT', 'UP', 'UP', 'DOWN', 'DOWN', 'RIGHT', 'RIGHT', 'LEFT', 'LEFT', 'DOWN', 'DOWN', 'UP', 'UP', 'UP', 'UP', 'DOWN', 'UP', 'DOWN', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'DOWN', 'DOWN', 'UP', 'UP', 'UP', 'DOWN', 'DOWN', 'DOWN', 'UP